In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import random
import string
from torch.utils.data import Dataset, DataLoader


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")                     # gpu availabi;ity
print(f"Using device: {device}")


df = pd.read_csv("/content/poems-100.csv")                               # loading poem
poems = df['text'].tolist()


def clean_text(text):                                                            # cleaning  text
    text = text.lower()  # Lowercase the text
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    return text


poems = [clean_text(poem) for poem in poems]                                          # tokenization
all_words = [word for poem in poems for word in poem.split()]
vocab = sorted(list(set(all_words)))
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for i, word in enumerate(vocab)}
vocab_size = len(vocab)


def one_hot_encode(word_idx, vocab_size):                                           # one hot encoding function
    one_hot = torch.zeros(vocab_size)
    one_hot[word_idx] = 1
    return one_hot


sequences = []
for poem in poems:                                                               ## Create sequences
    words = poem.split()
    for i in range(1, len(words)):
        sequences.append((words[i - 1], words[i]))


Using device: cuda


In [ ]:
class PoemDataset(Dataset):
    def __init__(self, sequences, word_to_idx, vocab_size):                        # preparing dataset
        self.sequences = sequences
        self.word_to_idx = word_to_idx
        self.vocab_size = vocab_size

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        x, y = self.sequences[idx]
        x_idx = self.word_to_idx[x]
        y_idx = self.word_to_idx[y]
        x_one_hot = one_hot_encode(x_idx, self.vocab_size)
        return x_one_hot, torch.tensor(y_idx, dtype=torch.long)

                                                                                        # Initialize dataset and dataloader
dataset = PoemDataset(sequences, word_to_idx, vocab_size)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [ ]:

class PoemGeneratorLSTM(nn.Module):                                                     # LSTM model
    def __init__(self, vocab_size, hidden_dim, num_layers=2):
        super(PoemGeneratorLSTM, self).__init__()
        self.lstm = nn.LSTM(vocab_size, hidden_dim, num_layers=num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden):
        output, hidden = self.lstm(x.unsqueeze(1), hidden)
        output = self.linear(output[:, -1, :])
        return output, hidden

# Model initialization
hidden_dim = 512
num_layers = 2
model = PoemGeneratorLSTM(vocab_size, hidden_dim, num_layers).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
epochs = 210
for epoch in range(epochs):
    total_loss = 0                                                         # training the model
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        hidden = None
        outputs, hidden = model(inputs, hidden)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}")

Epoch 1, Loss: 3.4892905561417495
Epoch 2, Loss: 3.4605012940619275
Epoch 3, Loss: 3.4337211396410057
Epoch 4, Loss: 3.4148065710314817
Epoch 5, Loss: 3.398494314035603
Epoch 6, Loss: 3.3784999316220454
Epoch 7, Loss: 3.36586990134086
Epoch 8, Loss: 3.3533110729770956
Epoch 9, Loss: 3.340236270983602
Epoch 10, Loss: 3.3334807336639245
Epoch 11, Loss: 3.318866882917177
Epoch 12, Loss: 3.3097597897979263
Epoch 13, Loss: 3.3049614454180465
Epoch 14, Loss: 3.295273131039476
Epoch 15, Loss: 3.2923369592953224
Epoch 16, Loss: 3.285185204886402
Epoch 17, Loss: 3.277173000296163
Epoch 18, Loss: 3.2710125779858523
Epoch 19, Loss: 3.267511255382874
Epoch 20, Loss: 3.262564681354582
Epoch 21, Loss: 3.259156341997453
Epoch 22, Loss: 3.2557169093986866
Epoch 23, Loss: 3.2543617011351906
Epoch 24, Loss: 3.2464275137748126
Epoch 25, Loss: 3.2425549437962666
Epoch 26, Loss: 3.239116403105345
Epoch 27, Loss: 3.2366106658401885
Epoch 28, Loss: 3.235249259928965
Epoch 29, Loss: 3.2290759864866425
Epoch 3

In [ ]:


# Generate poem function
def generate_poem_lstm(model, start_word, word_to_idx, idx_to_word, device, length=50):
    model.eval()
    generated_poem = [start_word]
    input_idx = word_to_idx[start_word]
    input_one_hot = one_hot_encode(input_idx, vocab_size).unsqueeze(0).to(device)
    hidden = None

    with torch.no_grad():
        for _ in range(length - 1):
            output, hidden = model(input_one_hot, hidden)
            probabilities = torch.softmax(output, dim=1).cpu().numpy()[0]
            predicted_word_idx = random.choices(range(len(probabilities)), weights=probabilities, k=1)[0]
            predicted_word = idx_to_word[predicted_word_idx]
            generated_poem.append(predicted_word)
            input_one_hot = one_hot_encode(predicted_word_idx, vocab_size).unsqueeze(0).to(device)

    formatted_poem = ""
    line = []
    word_count = 0
    for word in generated_poem:
        line.append(word)
        word_count += 1
        if word_count >= 7:
            formatted_poem += " ".join(line) + "\n"
            line = []
            word_count = 0
    if line:
        formatted_poem += " ".join(line)
    return formatted_poem

# Generate and print a poem
start_word = "sun"
generated_poem = generate_poem_lstm(model, start_word, word_to_idx, idx_to_word, device)
print("\nGenerated Poem:\n", generated_poem)


Generated Poem:
 sun there and with the calm once
coalmen us notgreat god for a while
what piping heart i will believe you
see the golden same little thou frigate
gale had long stretches as to a
young journey while he mile i long
and reachd breathe was the midst choir
of
